In [27]:
text = """Alice was beginning to get very tired of sitting by her sister on the bank, 
and of having nothing to do: once or twice she had peeped into the book her sister was reading, 
but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 
'without pictures or conversation?'"""

In [28]:
with open("./dataset.txt") as data_file:
    text = data_file.read()

In [29]:
text = text[:10_000]

In [30]:
import torch
import torch.nn as nn
import numpy as np

# Токенизация
words = text.lower().replace('\n', ' ').split()

# Создание словаря
word_to_idx = {word: idx for idx, word in enumerate(set(words))}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(word_to_idx)

# One-hot encoding
def one_hot(word, word_to_idx):
    vector = torch.zeros(vocab_size)
    vector[word_to_idx[word]] = 1
    return vector

# Пример:
# print(one_hot('alice', word_to_idx))

In [31]:
words

['я',
 'в',
 'париже:',
 'я',
 'начал',
 'жить,',
 'а',
 'не',
 'дышать.',
 'дмитриев.',
 'журнал',
 'путешественника.',
 'в',
 'числе',
 'молодых',
 'людей,',
 'отправленных',
 'петром',
 'великим',
 'в',
 'чужие',
 'края,',
 'для',
 'приобретения',
 'сведений,',
 'необходимых',
 'государству',
 'преобразованному,',
 'находился',
 'его',
 'крестник,',
 'арап',
 'ибрагим.',
 'он',
 'обучался',
 'в',
 'парижском',
 'военном',
 'училище,',
 'выпущен',
 'был',
 'капитаном',
 'артиллерии,',
 'отличился',
 'в',
 'испанской',
 'войне',
 'и,',
 'тяжело',
 'раненный,',
 'возвратился',
 'в',
 'париж.',
 'император',
 'посреди',
 'обширных',
 'своих',
 'трудов',
 'не',
 'преставал',
 'осведомляться',
 'о',
 'своем',
 'любимце',
 'и',
 'всегда',
 'получал',
 'лестные',
 'отзывы',
 'насчет',
 'его',
 'успехов',
 'и',
 'поведения.',
 'петр',
 'был',
 'очень',
 'им',
 'доволен',
 'и',
 'неоднократно',
 'звал',
 'его',
 'в',
 'россию,',
 'но',
 'ибрагим',
 'не',
 'торопился.',
 'он',
 'отговаривался'

In [32]:
seq_length = 5  # Длина входной последовательности
X = []
y = []

for i in range(len(words) - seq_length):
    input_seq = words[i:i+seq_length]
    target_word = words[i+seq_length]
    
    # Преобразуем в one-hot векторы
    input_seq_encoded = [one_hot(word, word_to_idx) for word in input_seq]
    target_word_encoded = one_hot(target_word, word_to_idx)
    
    X.append(input_seq_encoded)
    y.append(target_word_encoded)

# Преобразуем в тензоры
X = torch.stack([torch.stack(seq) for seq in X])
y = torch.stack(y)

# Пример:
# print(X.shape)  # (num_samples, seq_length, vocab_size)
# print(y.shape)  # (num_samples, vocab_size)

In [33]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x: (batch_size, seq_length, input_size)
        output, _ = self.lstm(x)
        # output: (batch_size, seq_length, hidden_size)
        output = self.fc(output[:, -1, :])  # Берем последний выход LSTM
        return output

# Параметры модели
input_size = vocab_size
hidden_size = 128
output_size = vocab_size
model = LSTMModel(input_size, hidden_size, output_size)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/150], Loss: 6.8114
Epoch [20/150], Loss: 6.6162
Epoch [30/150], Loss: 6.3736
Epoch [40/150], Loss: 6.2347
Epoch [50/150], Loss: 6.1408
Epoch [60/150], Loss: 6.0164
Epoch [70/150], Loss: 5.8640
Epoch [80/150], Loss: 5.6740
Epoch [90/150], Loss: 5.4185
Epoch [100/150], Loss: 4.9966
Epoch [110/150], Loss: 4.2697
Epoch [120/150], Loss: 3.3493
Epoch [130/150], Loss: 2.4484
Epoch [140/150], Loss: 1.6852
Epoch [150/150], Loss: 1.1239


In [35]:
def predict_next_word(input_seq):
    model.eval()
    with torch.no_grad():
        input_seq_encoded = torch.stack([one_hot(word, word_to_idx) for word in input_seq]).unsqueeze(0)
        output = model(input_seq_encoded)
        predicted_idx = torch.argmax(output, dim=1).item()
        return idx_to_word[predicted_idx]

# Пример:
input_seq = "Появление Ибрагима, его наружность, и".lower().split()
start_seq = input_seq.copy()

predicted = ['']

while len(predicted) < 4:
    predicted_word = predict_next_word(input_seq)
    predicted.append(predicted_word)
    input_seq.append(predicted_word)
    input_seq.pop(0)
    
print(f'Input: {" ".join(start_seq)}')
print(f'Output: {" ".join(predicted)}')

Input: появление ибрагима, его наружность, и
Output:  и природный ум


In [36]:
list(filter(lambda x: x.startswith("наружность"), word_to_idx.keys()))

['наружность,']